<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Market-Based Valuation of Equity Options

### Option Valuation Techniques Revisited &mdash; Complete Markets

Dr Yves J Hilpisch

The Python Quants GmbH

**CQF Lecture, 20. September 2018, London**

## Option Valuation

Different techniques are considered:

* discounting
* replication (linear algebra)
* approximation (OLS regression)
* neural nets (learning algorithm)
* martingale pricing (probabilistic approach)

## Most Simple Financial Market

In [1]:
import numpy as np

**_Two dates only (today, tomorrow) and two future states only with equal probability._**

### Traded Financial Assets

In [2]:
# risky stock -- future payoff
S = np.array((20., 5.))

In [3]:
# risky stock -- current price
S0 = 10.

In [4]:
# risk-less bond -- future payoff
B = np.array((11., 11.))

In [5]:
B0 = 10.

### The Market 

In [6]:
M = np.array((S, B)).T

In [7]:
M

array([[20., 11.],
       [ 5., 11.]])

In [8]:
M0 = np.array((S0, B0))

In [9]:
M0

array([10., 10.])

### A European Call Option 

In [10]:
# contingent claim -- future payoff
K = 15.
C = np.maximum(S - K, 0)

In [11]:
C

array([5., 0.])

## Returns

In [12]:
def ret(x, x0):
    return (x.mean() / x0) - 1

In [13]:
rS = ret(S, S0)
rS  # expected return

0.25

In [14]:
rB = ret(B, B0)
rB  # risk-less return ("sure thing")

0.10000000000000009

## Option Valuation &mdash; Wrong Way

In [15]:
C0w1 = C.mean() / (1 + rS)
C0w1

2.0

In [16]:
C0w2 = C.mean() / (1 + rB)
C0w2

2.2727272727272725

## Option Pricing &mdash; By Replication

In [17]:
# solving system of linear equations
phi = np.linalg.solve(M, C)

In [18]:
phi

array([ 0.33333333, -0.15151515])

In [19]:
np.dot(M, phi).round(2)

array([5., 0.])

In [20]:
C0 = np.dot(M0, phi)

In [21]:
C0

1.8181818181818188

## Volatility

In [22]:
def vol(x, x0):
    return (x / x0).std()

In [23]:
vol(S, S0)

0.75

In [24]:
vol(B, B0)

0.0

In [25]:
# volatility of option
vol(C, C0)

1.3749999999999996

In [26]:
# expected return of option
rC = ret(C, C0)
rC

0.37499999999999956

In [27]:
C.mean() / (1 + rC)

1.8181818181818188

## Option Pricing &mdash; By Regression

In [28]:
reg = np.linalg.lstsq(M, C, rcond=None)

In [29]:
reg

(array([ 0.33333333, -0.15151515]),
 array([], dtype=float64),
 2,
 array([24.96643328,  6.60887353]))

In [30]:
np.dot(M, reg[0]).round(2)

array([5., 0.])

In [31]:
np.dot(M0, reg[0])

1.8181818181818188

## Option Pricing &mdash; By Neural Network

<img src="http://hilpisch.com/images/neural_net.png">

In [32]:
# intial weights
w = np.array((1., -1.))

In [33]:
# layer one (output)
l1 = np.dot(M, w)
l1

array([ 9., -6.])

In [34]:
# deltas
d = l1 - C
d

array([ 4., -6.])

In [35]:
# mean squared error
(d ** 2).mean()

26.0

In [36]:
# learning rate
alpha = 0.05

In [37]:
# update values
u = alpha * d

In [38]:
u

array([ 0.2, -0.3])

In [39]:
# updating weights
w -= u

In [40]:
w

array([ 0.8, -0.7])

In [41]:
l1 = np.dot(M, w)
d = l1 - C
(d ** 2).mean()

12.29

In [42]:
l1

array([ 8.3, -3.7])

In [43]:
for _ in range(10):
    l1 = np.dot(M, w)
    d = l1 - C
    # print((d ** 2).mean())
    u = alpha * d
    w -= u
l1

array([ 5.07333548, -0.08783393])

In [44]:
w

array([ 0.34041119, -0.15999228])

In [45]:
np.dot(M0, w)

1.8041890736968753

## Martingale Measure

In [46]:
from scipy.optimize import minimize

In [47]:
def E(P):
    return np.dot(S, P) / (1 + rB) 

In [48]:
P = np.array((0.46, 0.54))

In [49]:
E(P)

10.81818181818182

In [50]:
E(P) - S0

0.8181818181818201

In [51]:
cons = ({'type': 'eq', 'fun': lambda Q: Q.sum() - 1})

In [52]:
bnds = ((0, 1), (0, 1))

In [53]:
opt = minimize(lambda Q: (E(Q) - S0) ** 2, (0.5, 0.5), constraints=cons,
                              bounds=bnds)

In [54]:
opt

     fun: 1.0172028584007344e-14
     jac: array([ 1.25849558e-06, -6.09001821e-07])
 message: 'Optimization terminated successfully.'
    nfev: 9
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([0.39999999, 0.60000001])

In [55]:
Q = opt['x']

In [56]:
Q

array([0.39999999, 0.60000001])

In [57]:
E(Q) - S0

-1.0085647517144025e-07

## Martingale Pricing

In [58]:
np.dot(C, Q) / (1 + rB)

1.8181817845629937

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>

**Training** |
<a href="http://training.tpq.io">http://training.tpq.io</a>

**Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>